In [3]:
import os
from dotenv import load_dotenv, find_dotenv

import httpx
import pandas as pd

load_dotenv(find_dotenv())

True

In [4]:
def get_token():
    response = httpx.post(
        str(os.getenv("TOKEN_ENDPOINT")),
        data={
            "client_id": str(os.getenv("CLIENT_ID")),
            "client_secret": str(os.getenv("CLIENT_SECRET")),
            "scope": str(os.getenv("SCOPE")),
            "grant_type": "client_credentials",
        },
        proxies={
            "http://": httpx.Proxy(str(os.getenv("FORD_PROXY"))),
            "https://": httpx.Proxy(str(os.getenv("FORD_PROXY"))),
        },
        timeout=160,
    )
    print(
        "token expires in:",
        round(int(response.json()["expires_in"]) / 60, 0),
        "minutes",
    )
    return response.json()["access_token"]

In [5]:
def call_api_movement(prompt: str, token: str):
    content = {
        "model": "gpt-4",
        "context": "Analyse this complaint and answer this following question",  # sets the overall behavior of the assistant.
        "messages": [
            {
                "role": "user",
                "content": prompt,
            }
        ],
        "parameters": {
            "temperature": 0.05,  # Determines the randomnes of the model's response.
        },
    }
    try:
        response = httpx.post(
            str(os.getenv("API_ENDPOINT")),
            headers={"Authorization": f"Bearer {token}"},
            json=content,
            timeout=20,
        )
        message = response.json()["content"]
        print(message)
        if "\n" in message:
            message = message.split("\n")[0]

        return message

    except httpx.HTTPError as exc:
        print(exc)

### DO YOUR ANALYSIS BEYOND THIS COMMENT 

In [6]:
df = pd.read_excel("CLMLC135.xlsm", header=3, sheet_name="Sheet1")
df["to be binned"] = (
    "Customer comment: "
    + df["Customer Comments (English)"]
    + ". Technician Comments (native language): "
    + df["Technician Comments"]
    + " English Technician comments: "
    + df["Tech Comments (English)"]
)

In [10]:
token = get_token()
content = (
    f"Please provide aswer to the question 'is the following complaint i"
    + "s related to rear window defrost troubles ?'. This could include "
    + "issues such as defrost disconnect, defrost not working, defrost c"
    + "racking the window, defrost exploding the window, defrost poor wo"
    + "rking, defrost being very hot and all kind of defrost problem. Th"
    + "e complaints are in English, Portuguese, and Spanish. ONLY ANSWER"
    + "YES OR NO IN CAPITAL LETTERS, DON'T CHANGE THIS FORMAT. Aswer for"
    + f" the following complaint: "
)
df["filter"] = df["to be binned"].apply(lambda row: call_api_movement(row, token))

df[df["filter"] == "YES"].to_csv("Result.csv")

token expires in: 60.0 minutes
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
YES
YES
NO
NO
NO
NO
YES
YES
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
YES
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
YES
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO


KeyError: 'content'